In [33]:
import numpy as np
from scipy import interpolate
import time
import math
import matplotlib.pyplot as plt
# from misc.golden_section_search import Search

In [34]:
"""
Michael Pham
February 2025

much thanks, heavy inspiration from Prof. Heer from Ch. 10.1.2 in Heer/Maussner, Dynamic General Equilibrium Modeling: Computational
Methods and Applications, Algorithm 10.1.1

version 4 with K, Z, H, L (solved endogenously) state variable
no consumption tax
"""

'\nMichael Pham\nFebruary 2025\n\nmuch thanks, heavy inspiration from Prof. Heer from Ch. 10.1.2 in Heer/Maussner, Dynamic General Equilibrium Modeling: Computational\nMethods and Applications, Algorithm 10.1.1\n\nversion 4 with K, Z, H, L (solved endogenously) state variable\nno consumption tax\n'

In [35]:
exp = np.e
log = math.log

In [36]:
# Parameters - households and firms
beta = 0.935  # discount factor
eta = 2.0    # coefficient of relative risk aversion
gamma = 2  # disutility from working
alpha = 0.33 # capital share in production
delta = 0.1  # depreciation rate
# omega = 0.1  # parent education time factor
phi_p = 0.324  # share of parental input in education
psi_e = 0.29  # CES parameter for education inputs
theta_I = 0.5  # human capital investment effectiveness
varsigma_g = 1
varsigma_p = 1


# phi_p = 0.324
# varsigma_p = 1
# varsigma_g = 1
# theta_I = 0.5
# psi_e = 0.29
# I_e = (0.06 * Y)
# I_e_mean = (0.035*Y)
# e_mean = 0.01
# e_min = 0.001
# e_max = 1





tfp = 1      #total factor productivity
small = 0.00001
neg = -1e10

#Parameters - government
tau = 0.15
Ie_gdp = 0.06 # public education investment as percent of GDP
Iz_gdp = 0.06 # public infrastructure investment as percent of GDP percent of GDP
sigma_bar = 0.3  # public investment inefficiency
G_gdp = 0.07      # grants (as a fraction of GDP)
psi = 0.1   # public infrastructure elasticity
# nu = 0.6     # debt-to-GDP ratio limit

# Initial guesses
# K_init = 1
# H_init = 1
# L_init = 0.2
# Z_init = 1
# tau_init = 0.15  # Initial guess for tax rate
# T_init = 0.1  # Initial guess for transfers
# D_init = 0

# Grid spec
na = 4  # number of points in capital grid
nh = 4   # number of points in human capital grid
# nZ = 20   # number of points in public infrastructure grid
# Kmin = 0
# Kmax = 10
# hmin = 0
# hmax = 10
# Zmin = 0
# Zmax = 10

k_start = 0
k_min, k_max = 0, 2
k_grid =  np.linspace(k_min, k_max, na)   # asset grid 
k_eps = (k_grid[1]-k_grid[0])/na    #  test for corner solution
phi = 0.8           # updating of the aggregate capital stock K in outer loop
tolerance = 0.001        # percentage deviation of final solution 
tolerance_gs = 1e-10       # tolerance for golden section search 
max_iters = 30             # maximum number of iteration over K


# h_min, h_max = 1, 1500
h_min, h_max = 1, 10


# Number of periods
periods = 6
child_periods = 4

In [37]:
l_init = 0.2
r_init = beta**(-1) - 1

Production Equations

In [38]:
def production(Z_prev, K_prev, H, L):
    """
    production function (eq 1 mod)

    params: K_t-1, H_t, L_t
    returns: Y
    """
    return tfp * Z_prev**psi * K_prev**(alpha-psi) * (H*L)**(1-alpha-psi)

def wage_rate(Z_prev, K_prev, H, L):
    """
    marginal product of labor (eq 2 mod)

    params: K_t-1, H_t, L_t
    returns: w_t
    """
    return tfp * Z_prev**psi * (1-alpha-psi) * (K_prev)**(alpha-psi) * (H*L)**(-alpha-psi)

def interest_rate(Z_prev, K_prev, H, L):
    """
    marginal product of capital minus depreciation (eq 3 mod + interest steady state)

    params: K_t-1, H_t, L_t
    returns: r_t 
    """
    return tfp * (alpha -psi) * Z_prev**psi * (H*L)**(1-alpha-psi) * K_prev**(alpha-1-psi) - delta

Human Capital Equations

In [39]:
def human_capital_formation(h_c, e_p, I_e, e_mean, I_e_mean):
    """

    params: 
    returns: 
    """
    I_c = (phi_p * (varsigma_p * (e_p / e_mean)) ** psi_e + (1-phi_p) * (varsigma_g * I_e/I_e_mean) ** psi_e ) ** (1/psi_e)
    
    return (I_c ** theta_I) * (h_c ** (1-theta_I))

Government Equations

In [40]:
def transfer(K_prev, H, L, G, tau, w, r):
  """
  government budget (eq 6 mod)
  assumes balanced budget, no borrowing

  params: K_t-1, H_t, L_t, G, tau, w_t, r_t
  returns: T (transfers)
  """
  tax_revenue = tau*w*H*L + tau*r*K_prev
  total_revenue = tax_revenue + G
  return total_revenue #- Y*Ie_gdp

Household Equations

In [41]:
def utility(c, l):
  """
  household utility function (eq 8)
  adds modified case of when full labor, so utility is not 0

  params: c_t, l_t
  returns: utility
  """

  if l == 1:
    return np.log(c+ small) + gamma * np.log(l)

  return (((c+small) * (1-l) ** gamma) ** (1-eta) - 1) / (1-eta)


# def next_value_function(k):
#   return vr_polate(k)

def labor_foc(k, k_prev, h, w, r, tau, T):

  l = (1/(1+gamma)) * ( 1 - (gamma * (((1+r) * k_prev - tau * r * k_prev + T - k))/((1-tau)*w*h)))

  if l < 0: # corner solution 0<=n<=1
    l = 0
  elif l > 1: # corner solution
    l = 1

  return l


def value_function(next_value_interpolator, k, k_prev, h, w, r, tau, T):
  """
  household bellman equation (eq 14)
  
  params: k_t, k_t-1, c_t, l_t, h_t, w_t, r_t, tau, T
  returns: value of this period's bellman equation
  """
  l = labor_foc(k, k_prev, h, w, r, tau, T)


  c = (1-tau)*w*h*l - k + (1+r)*k_prev - tau*r*max(k_prev, 0) + T
  if c <= 0:
    return neg
  # print(f"in value function, c: {c}, l, {l}, interpolator, {next_value_interpolator}")
  return utility(c, l) + beta * next_value_interpolator(k)

def consumption(k_prev, k, h, l, w, r, tau, T):
  return (1-tau)*w*h*l + (1+ r)*k_prev - tau*r*max(k_prev, 0) + T - k


def final_child_value(hc):
  return 50 * hc


def value_function_parent_final(next_value_interpolator, k, k_prev, h, w, r, tau, T, hc, I_e, e_mean, I_e_mean):
  """
  household bellman equation (eq 14)
  
  params: k_t, k_t-1, c_t, l_t, h_t, w_t, r_t, tau, T
  returns: value of this period's bellman equation
  """
  l = labor_foc(k, k_prev, h, w, r, tau, T)

  # print(f"hc = {hc}")

  c_total = (1-tau)*w*h*l - k + (1+r)*k_prev - tau*r*max(k_prev, 0) + T
  if c_total <= 0:
    return neg
  
  #do GSS to find e*
  # Golden section search to find optimal c
  a = 0  # minimum consumption
  b = c_total  # maximum consumption
  rg = (3.0 - np.sqrt(5.0)) / 2.0  # golden ratio
  
  c = a + rg * (b - a)
  e_c = c_total - c
  hc_next_c = human_capital_formation(h_c=hc, e_p=e_c, I_e=I_e, e_mean=e_mean, I_e_mean=I_e_mean)
  # print(f"hc_next_c = {hc_next_c}")
  val_c = utility(c, l) + beta * next_value_interpolator(k) + beta * final_child_value(hc_next_c)
  # print(f"utility  = {utility(c, l)}")
  # print(f"nexthc {final_child_value(hc_next_c)}") #errored
  # print(f"next_val {next_value_interpolator(k)}")
  # print(f"val_c {val_c}")

  d = a + (1.0 - rg) * (b - a)
  e_d = c_total - d
  hc_next_d = human_capital_formation(h_c=hc, e_p=e_d, I_e=I_e, e_mean=e_mean, I_e_mean=I_e_mean)
  val_d = utility(d, l) + beta * next_value_interpolator(k) + beta * final_child_value(hc_next_d)
  # print(f"val_d {val_d}")



  tol = 1e-6
  diff = 1e4
  
  while diff > tol:
      if val_c >= val_d:
          b = d
          d = c
          val_d = val_c
          c = a + rg * (b - a)
          e_c = c_total - c
          hc_next_c = human_capital_formation(h_c=hc, e_p=e_c, I_e=I_e, e_mean=e_mean, I_e_mean=I_e_mean)
          val_c = utility(c, l) + beta * next_value_interpolator(k) + beta * final_child_value(hc_next_c)
      else:
          a = c
          c = d
          val_c = val_d
          d = a + (1.0 - rg) * (b - a)
          e_d = c_total - d
          hc_next_d = human_capital_formation(h_c=hc, e_p=e_d, I_e=I_e, e_mean=e_mean, I_e_mean=I_e_mean)
          val_d = utility(d, l) + beta * next_value_interpolator(k) + beta * final_child_value(hc_next_d)
      
      diff = b - a
  
  c = (a + b) / 2.0
  e = c_total - c
  hc_next = human_capital_formation(h_c=hc, e_p=e, I_e=I_e, e_mean=e_mean, I_e_mean=I_e_mean)
  value = utility(c, l) + beta * next_value_interpolator(k) + beta * final_child_value(hc_next)
  # print(f"in value function, c: {c}, l, {l}, interpolator, {next_value_interpolator}")
  return value




def value_function_parent(next_value_interpolator, k, k_prev, h, w, r, tau, T, hc, I_e, e_mean, I_e_mean):
  """
  household bellman equation (eq 14)
  
  params: k_t, k_t-1, c_t, l_t, h_t, w_t, r_t, tau, T
  returns: value of this period's bellman equation
  """
  l = labor_foc(k, k_prev, h, w, r, tau, T)



  c_total = (1-tau)*w*h*l - k + (1+r)*k_prev - tau*r*max(k_prev, 0) + T
  if c_total <= 0:
    return neg
  
  #do GSS to find e*
  # Golden section search to find optimal c
  a = 0  # minimum consumption
  b = c_total  # maximum consumption
  rg = (3.0 - np.sqrt(5.0)) / 2.0  # golden ratio
  
  c = a + rg * (b - a)
  e_c = c_total - c
  hc_next_c = human_capital_formation(h_c=hc, e_p=e_c, I_e=I_e, e_mean=e_mean, I_e_mean=I_e_mean)
  val_c = utility(c, l) + beta * next_value_interpolator(k) + beta * final_child_value(hc_next_c)
  
  d = a + (1.0 - rg) * (b - a)
  e_d = c_total - d
  hc_next_d = human_capital_formation(h_c=hc, e_p=e_d, I_e=I_e, e_mean=e_mean, I_e_mean=I_e_mean)
  val_d = utility(d, l) + beta * next_value_interpolator(k) + beta * final_child_value(hc_next_d)
  
  tol = 1e-6
  diff = 1e4
  
  while diff > tol:
      if val_c >= val_d:
          b = d
          d = c
          val_d = val_c
          c = a + rg * (b - a)
          e_c = c_total - c
          hc_next_c = human_capital_formation(h_c=hc, e_p=e_c, I_e=I_e, e_mean=e_mean, I_e_mean=I_e_mean)
          val_c = utility(c, l) + beta * next_value_interpolator(k) 
      else:
          a = c
          c = d
          val_c = val_d
          d = a + (1.0 - rg) * (b - a)
          e_d = c_total - d
          hc_next_d = human_capital_formation(h_c=hc, e_p=e_d, I_e=I_e, e_mean=e_mean, I_e_mean=I_e_mean)
          val_d = utility(d, l) + beta * next_value_interpolator(k) 
      
      diff = b - a
  
  c = (a + b) / 2.0
  e = c_total - c
  hc_next = human_capital_formation(h_c=hc, e_p=e, I_e=I_e, e_mean=e_mean, I_e_mean=I_e_mean)
  value = utility(c, l) + beta * next_value_interpolator(k) 
  # print(f"in value function, c: {c}, l, {l}, interpolator, {next_value_interpolator}")
  return value

In [42]:
h_init = 2
l_init = 0.3

#initialize steady state variable predictions
L_bar = l_init #refers to guess of L in this period
H_bar = h_init #refers to guess of H in this period
Z_bar = 0.1
K_bar = ((r_init + delta)/((alpha-psi)*tfp*(Z_bar**psi)))**(1/(alpha-psi-1)) * (H_bar * L_bar) #new guess!
L_old = L_bar + 2
H_old = H_bar + 100
K_old = K_bar + 100
Z_old = Z_bar + 100

# h = 3
l = 0.2

K_bar

0.6612600167995938

In [43]:
def GSS(next_value_interpolator, function, left, initial_guess, right, tolerance, k_prev, h, w, r, tau, T):
    """
    Find the maximum of a function using golden section search.
    
    Args:
        next_value_interpolator: Function to interpolate next period's value
        function: Function to maximize
        left: Left bound of search interval
        initial_guess: Initial guess for the maximum
        right: Right bound of search interval
        tolerance: Convergence tolerance
        k_prev: Previous period's capital
        h: Human capital
        w: Wage
        r: Interest rate
        tau: Tax rate
        T: Transfer
        
    Returns:
        The value of k that maximizes the function
    """
    def evaluate_function(k):
        return function(next_value_interpolator, k, k_prev, h, w, r, tau, T)
    
    golden_ratio = ((1+np.sqrt(5))/2)
    golden_ratio_dec = golden_ratio - 1

    if abs(right - initial_guess) >= abs(initial_guess - left):
        golden_left = initial_guess
        golden_right = initial_guess + (1 - golden_ratio_dec) * (right - initial_guess)
    else:
        golden_right = initial_guess
        golden_left = initial_guess - (1 - golden_ratio_dec) * (initial_guess - left)

    while (right - left) > tolerance:
        if evaluate_function(golden_left) < evaluate_function(golden_right):
            left = golden_left
        elif evaluate_function(golden_left) > evaluate_function(golden_right):
            right = golden_right
        elif evaluate_function(golden_left) == evaluate_function(golden_right):
            left = golden_left
            right = golden_right

        golden_left = left + (1 - golden_ratio_dec) * (right-left)
        golden_right = left + (golden_ratio_dec) * (right-left)
    
    return left if evaluate_function(left) > evaluate_function(right) else right

In [44]:
def GSS2(next_value_interpolator, function, left, initial_guess, right, tolerance, k_prev, h, w, r, tau, T, hc, I_e, e_mean, I_e_mean):
    """
    Find the maximum of a function using golden section search.
    
    Args:
        next_value_interpolator: Function to interpolate next period's value
        function: Function to maximize
        left: Left bound of search interval
        initial_guess: Initial guess for the maximum
        right: Right bound of search interval
        tolerance: Convergence tolerance
        k_prev: Previous period's capital
        h: Human capital
        w: Wage
        r: Interest rate
        tau: Tax rate
        T: Transfer
        
    Returns:
        The value of k that maximizes the function
    """
    def evaluate_function(k):
        return function(next_value_interpolator, k, k_prev, h, w, r, tau, T, hc, I_e, e_mean, I_e_mean)
    
    golden_ratio = ((1+np.sqrt(5))/2)
    golden_ratio_dec = golden_ratio - 1

    if abs(right - initial_guess) >= abs(initial_guess - left):
        golden_left = initial_guess
        golden_right = initial_guess + (1 - golden_ratio_dec) * (right - initial_guess)
    else:
        golden_right = initial_guess
        golden_left = initial_guess - (1 - golden_ratio_dec) * (initial_guess - left)

    while (right - left) > tolerance:
        if evaluate_function(golden_left) < evaluate_function(golden_right):
            left = golden_left
        elif evaluate_function(golden_left) > evaluate_function(golden_right):
            right = golden_right
        elif evaluate_function(golden_left) == evaluate_function(golden_right):
            left = golden_left
            right = golden_right

        golden_left = left + (1 - golden_ratio_dec) * (right-left)
        golden_right = left + (golden_ratio_dec) * (right-left)
    
    return left if evaluate_function(left) > evaluate_function(right) else right

In [45]:
def loggrid(xmin, xmax, n, degree) :
    """
    Create a grid with logarithmically spaced points between xmin and xmax.
    
    Args:
        xmin: Minimum value of the grid
        xmax: Maximum value of the grid
        n: Number of points in the grid
        degree: Parameter controlling the spacing density (default=1.0)
    
    Returns:
        np.ndarray: Array of logarithmically spaced points
    """

    # Create evenly spaced points between 0 and 1
    points = np.linspace(0, 1, n)
    
    # Transform to logarithmic spacing and apply degree
    # Handle the case where xmin is zero to avoid division by zero
    if xmin == 0:
        # Use linear spacing if xmin is zero
        grid = xmin + (xmax - xmin) * (points**degree)
    else:
        grid = xmin * (xmax/xmin)**(points**degree)

    return grid
    

In [46]:
def spbas(r: int, knots: np.ndarray) -> np.ndarray:
    """
    Evaluate basis function of cubic spline on r+2 knots.
    
    Args:
        r: Integer number of internal knots
        knots: Array of knot positions of length r+2
        
    Returns:
        invT: Inverse of the tridiagonal matrix T
    """
    # Calculate distances between consecutive knots
    dt = knots[1:] - knots[:-1]  # equivalent to dt(i) = knots(i+1)-knots(i)
    
    # Calculate special end conditions
    w1 = dt[1] - (dt[0]**2 / dt[1])
    wr = dt[r-1] - (dt[r]**2 / dt[r-1])
    
    # Initialize tridiagonal matrix T
    T = np.zeros((r, r))
    
    # Fill interior points of tridiagonal matrix
    for i in range(1, r-1):  # equivalent to do i = 2,r-1
        T[i, i-1] = dt[i+1]      # subdiagonal
        T[i, i] = 2.0 * (dt[i] + dt[i+1])  # diagonal
        T[i, i+1] = dt[i]        # superdiagonal
    
    # Set knot-a-knot condition entries
    T[0, 0] = 2.0 * (dt[0] + dt[1]) - w1
    T[0, 1] = dt[0] + (dt[0]**2 / dt[1])
    T[r-1, r-1] = 2.0 * (dt[r-1] + dt[r]) - wr
    T[r-1, r-2] = dt[r] + (dt[r]**2 / dt[r-1])
    
    # Return inverse of T
    return np.linalg.inv(T)

In [ ]:
hc_min = np.ones(child_periods+1)
hc_max = np.ones(child_periods+1)
e_mean = 0.02 * np.ones(child_periods+1)
e_minprop = 0.001
e_maxprop = 1
deggridh = 2.5
deggridk = 3.0

hc_grid = np.zeros((nh, child_periods + 1))


iteration = -1
K_diff = 1 + tolerance
state_var_iter_array = np.zeros((max_iters,4))

omega = 0.1


v_optimal_array = np.zeros((na, nh, periods + 1)) #vw initialize the agents' value function array
k1_optimal_array = np.zeros((na, nh, periods)) #initialize the optimal capital array
c_optimal_array = np.zeros((na, nh, periods)) #initialize the optimal consumption array
l_optimal_array = np.zeros((na, nh, periods)) #initialize the optimal labor array
h_optimal_array = np.zeros((na, nh, periods)) #initialize the optimal child human capital array
hc_optimal_array = np.zeros((na, nh, child_periods)) #initialize the optimal child human capital array


v3d_optimal_array = np.zeros((na, nh, periods + 1, nh))
k13d_optimal_array = np.zeros((na, nh, periods, nh))

max_iters_ss = max_iters
# tolerance_ss = 0.01

#loop while max_iters not exceeded and difference between this K guess and previous K guess is greater than acceptable tolerance
while iteration < max_iters - 1 and K_diff > tolerance: #TODO fix k tolerance
  iteration += 1
  K_diff = abs((K_bar - K_old) / K_bar)
  H_diff = abs((H_bar - H_old) / H_bar) #can you solve for H in terms of K?
  L_diff = abs((L_bar - L_old) / L_bar)
  Z_diff = abs((Z_bar - Z_old) / Z_bar)
  print(f"iter: {iteration}; K change: {K_diff:.2f}; H change: {H_diff:.2f}; L change: {L_diff:.2f}; Z change: {Z_diff:.2f}")

  #calculate variables based on new guess of K, H, L
  w = wage_rate(Z_prev = Z_bar, K_prev = K_bar, H = H_bar, L = L_bar)
  r = interest_rate(Z_prev = Z_bar, K_prev = K_bar, H = H_bar, L = L_bar)
  Y = production(Z_prev = Z_bar, K_prev = K_bar, H = H_bar, L = L_bar)
  grants = G_gdp * Y
  Iz = Iz_gdp * Y
  Ie = Ie_gdp * Y
  I_e_mean = (0.03*Y)

  print(f"y: {Y}")
  print(f"ie: {Ie}")

  tax_revenue = tau*w*H_bar*L_bar + tau*r*K_bar
  print(f"tax_revenue: {tax_revenue}")
  print(f"grants: {grants}")
  T = transfer(K_prev = K_bar, H = H_bar, L = L_bar, G = grants, tau = tau, w = w, r = r) - Iz - Ie


  #SHOULD THIS BE HERE OR END?
  print(f"T: {T}")
  print(f"iter: {iteration}; K: {K_bar:.2f}; H: {H_bar:.2f}; L: {L_bar:.2f}; Z: {Z_bar:.2f}")
  print(f"iter: {iteration}; Y: {Y:.2f}; tau: {tau:.2f}; T: {T:.2f}")
  K_old = K_bar
  H_old = H_bar
  L_old = L_bar
  Z_old = Z_bar
  state_var_iter_array[iteration, 0] = K_old #TODO: update these each iteration
  state_var_iter_array[iteration, 1] = H_old
  state_var_iter_array[iteration, 2] = L_old
  state_var_iter_array[iteration, 3] = Z_old






  #FIRST ITER ONLY
  if iteration == 0:
    #minimun hc vectors
    for i in range(1, child_periods + 1): 
      hc_min[i] = human_capital_formation(h_c = hc_min[i-1], e_p = e_mean[i]*e_minprop, I_e = Ie, e_mean = e_mean[i], I_e_mean = I_e_mean)

      hc_min[i] = human_capital_formation(h_c = hc_min[i-1], e_p = e_mean[i]*e_minprop, I_e = Ie, e_mean = e_mean[i], I_e_mean = I_e_mean)

    #maximun hc vectors
    for i in range(1, child_periods + 1):
      hc_max[i] = human_capital_formation(h_c = hc_max[i-1], e_p = e_mean[i]*e_maxprop, I_e = Ie, e_mean = e_mean[i], I_e_mean = I_e_mean)



    # print("hc_min:", hc_min)
    # print("hc_max:", hc_max)

    # hc_minval = min(hc_min[1:])
    # hc_maxval = max(hc_max[1:])


    #see 402-404 #UNCOMMENT LATER
    # dampssbfinal = speedbeta
    # dampssw1final = dampssw
    # dampssw2final = dampssw

  # h_grid = loggrid(hc_minval[child_periods-1], min(h_max, hc_maxval[child_periods-1]), nh, deggridh)
  # h_grid = loggrid(hc_min[child_periods], min(h_max, hc_max[child_periods]), nh, deggridh)
  h_grid = loggrid(hc_min[child_periods], max(h_max, hc_max[child_periods]), nh, deggridh)

    # print(f"h_grid: {h_grid}")
  k_grid = loggrid(k_min, k_max, na, deggridk)
    # print(f"h_grid: {k_grid}")


  for i in range(child_periods + 1):
      hc_grid[:,i] = loggrid(hc_min[i], hc_max[i], nh, deggridh)
      # print(f"period {i}")
      # print(hc_grid[:,i])


  invTa = spbas(na - 2, k_grid)
    # print(invTa)

  invTh = spbas(nh - 2, h_grid)

  for i in range(child_periods):
      invThcall = np.ones((nh - 2, nh -2, child_periods))
      z = spbas(nh - 2, hc_grid[:,i])
      # print(z)
      invThcall[:,:,i] = z

  # print(invTa)
  # print(invThcall[:,:,2])
    # break #427
    # compute last period value function st eq. 15 
    # FIX THIS LATER

  #last period
  for ik in range(na):
    for ih in range(nh):
      l_optimal_array[ik, ih, periods - 1] = labor_foc(k = 0, k_prev = k_grid[ik], h = h_grid[ih], w = w, r = r, tau = tau, T = T) 
      final_period_assets = (1-tau)*w*h_grid[ih]*l_optimal_array[ik, ih, periods - 1] + (1+r)*k_grid[ik] - tau*r*max(k_grid[ik], 0) + T
      v_optimal_array[ik, ih, periods - 1] = utility(final_period_assets, l_optimal_array[ik, ih, periods - 1])
      c_optimal_array[ik, ih, periods - 1] = final_period_assets
      # h_optimal_array[ik, ih, periods - 1] = h
      k1_optimal_array[ik, ih, periods - 1] = 0

    #looking at j = 8
  # for ik in range(na):
  #   for ih in range(nh):

    



  # print(f"T: {T}")
  # print(f"iter: {iteration}; K: {K_bar:.2f}; H: {H_bar:.2f}; L: {L_bar:.2f}; Z: {Z_bar:.2f}")
  # print(f"iter: {iteration}; Y: {Y:.2f}; tau: {tau:.2f}; T: {T:.2f}")
  # K_old = K_bar
  # H_old = H_bar
  # L_old = L_bar
  # Z_old = Z_bar
  # state_var_iter_array[iteration, 0] = K_old #TODO: update these each iteration
  # state_var_iter_array[iteration, 1] = H_old
  # state_var_iter_array[iteration, 2] = L_old
  # state_var_iter_array[iteration, 3] = Z_old


  


  # #compute last period value function st eq. 15
  # for i in range(na):
  #   l_optimal_array[i, periods - 1] = labor_foc(k = 0, k_prev = k_grid[i], h = h, w = w, r = r, tau = tau, T = T) #ADDednfoc
  #   final_period_assets = (1-tau)*w*h*l_optimal_array[i, periods - 1] + (1+r)*k_grid[i] - tau*r*max(k_grid[i], 0) + T
  #   value_array[i, periods - 1] = utility(final_period_assets, l_optimal_array[i, periods - 1])
  #   c_optimal_array[i, periods - 1] = final_period_assets
  #   h_optimal_array[i, periods - 1] = h
  #   k_optimal_array[i, periods - 1] = 0
  #second to last to 
  for i in range(periods-1, child_periods, -1): #loop backwards from age 60 to 0
    for ih in range(nh):
      vr_polate = interpolate.interp1d(k_grid, v_optimal_array[:,ih,i], kind='cubic', fill_value='extrapolate') #interpolator which lines up k_grid with value_array row i
      h0 = h_grid[ih]
      m0 = 0

      for ik in range(na): #iteration through all possible kt-1 assets could come into periods with
        # print(f"backwards iter: {iteration}, solving for age: {i}, k_t-1: {k_grid[ik]}, h_t: {h_grid[ih]}")

        k0 = k_grid[ik]

        ax = 0
        bx = -1
        cx = -2
        v0 = neg
        m = max(-1, m0-2)

        while (ax > bx or bx > cx):
          m += 1
          v1 = value_function(vr_polate, k = k_grid[m], k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T)

          if v1 > v0:
            if m == 0:
              ax = k_grid[m]
              bx = k_grid[m]

            else:
              bx = k_grid[m]
              ax = k_grid[m-1]

            v0 = v1
            m0 = m

          else:
            cx = k_grid[m]

          if m==na-1:
            ax = k_grid[m-1]
            bx = k_grid[m] 
            cx = k_grid[m] 
          

        if ax==bx:  # corner solution: a'=0?
            # w = wage_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
            # r = interest_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
            # T = transfer(K_prev = k_grid[m], H = H_bar, L = L_bar, G = grants, tau = tau, w = w, r = r)
            if value_function(vr_polate, k = ax, k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T) > value_function(vr_polate, k = k_eps, k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T):
              k1_optimal_array[ik, ih, i-1]=0
            else:
              # k_optimal_array[j,i-1] = Search(value_function, left=ax, initial_guess=k_eps, right=cx, tolerance=tolerance_gs, k_prev=k0, l=l, h=h, w=w, r=r, tau=tau, T=T)
              k1_optimal_array[ik, ih, i-1] = GSS(vr_polate, value_function_parent_final, ax, k_eps, cx, tolerance_gs,
                  k_prev=k0, h=h0, w=w, r=r, tau=tau, T=T)
                  
        elif bx==cx:  # corner solution: a'=a[na-1]=kmax?
            # w = wage_rate(K_prev = k_grid[na-1], H = H_bar, L = L_bar)
            # r = interest_rate(K_prev = k_grid[na-1], H = H_bar, L = L_bar)
            # T = transfer(K_prev = k_grid[na-1], H = H_bar, L = L_bar, G = grants, tau = tau, w = w, r = r)
            if value_function(vr_polate, k = k_grid[na-1], k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T) > value_function(vr_polate, k = k_grid[na-1] - k_eps, k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T):
              k1_optimal_array[ik, ih, i-1] = k_grid[na-1]
            else:
              # k_optimal_array[j,i-1] = Search(value_function, left=k_grid[na-2], initial_guess=k_max-k_eps, right=k_max, tolerance=tolerance_gs,
              #     k_prev=k0, l=l, h=h, w=w, r=r, tau=tau, T=T)  
              k1_optimal_array[ik, ih, i-1] = GSS(vr_polate, value_function, k_grid[na-2], k_max-k_eps, k_max, tolerance_gs,
                  k_prev=k0, h=h0, w=w, r=r, tau=tau, T=T)     
          
        else:
            # w = wage_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
            # r = interest_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
            # T = transfer(K_prev = k_grid[m], H = H_bar, L = L_bar, G = grants, tau = tau, w = w, r = r)
            # k_optimal_array[j,i-1] = Search(value_function, left=ax, initial_guess=bx, right=cx, tolerance=tolerance_gs,
            #       k_prev=k0, l=l, h=h, w=w, r=r, tau=tau, T=T)
            k1_optimal_array[ik,ih, i-1] = GSS(vr_polate, value_function, ax, bx, cx, tolerance_gs,
                  k_prev=k0, h=h0, w=w, r=r, tau=tau, T=T)
            
              

        k1 = k1_optimal_array[ik, ih, i-1]
        v_optimal_array[ik, ih, i-1] = value_function(vr_polate, k1_optimal_array[ik, ih, i-1], k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T)
      # l_optimal_array[ik, ih, i-1] = labor_foc(k = k1, k_prev = k0, h = h, w = w, r = r, tau = tau, T = T)
      # c_optimal_array[ik,ih, i-1] = (1-tau)*w*h*l_optimal_array[ik, ih, i-1] + r*k0 - tau*r*max(k0, 0) + T - k1
      # # print(f"iter: {iteration}, i: {i}, k opt: {k1}")

  #compute policy function for period 4 adult
  for i in range(child_periods, child_periods-1, -1): #loop backwards from age 60 to 0
    for ihc in range(nh):
      for ih in range(nh):
        vr_polate = interpolate.interp1d(k_grid, v_optimal_array[:,ih,i], kind='cubic', fill_value='extrapolate') #interpolator which lines up k_grid with value_array row i
        h0 = h_grid[ih]
        hc0 = h_grid[ihc]
        m0 = 0

        for ik in range(na): #iteration through all possible kt-1 assets could come into periods with
          # print(f"backwards iter: {iteration}, solving for age: {i}, k_t-1: {k_grid[ik]}, h_t: {h_grid[ih]}")

          k0 = k_grid[ik]

          ax = 0
          bx = -1
          cx = -2
          v0 = neg
          m = max(-1, m0-2)

          while (ax > bx or bx > cx):
            m += 1
            v1 = value_function_parent_final(next_value_interpolator = vr_polate, k = k_grid[m], k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T, hc = hc0, I_e = Ie, e_mean = e_mean[0], I_e_mean = I_e_mean)

            if v1 > v0:
              if m == 0:
                ax = k_grid[m]
                bx = k_grid[m]

              else:
                bx = k_grid[m]
                ax = k_grid[m-1]

              v0 = v1
              m0 = m

            else:
              cx = k_grid[m]

            if m==na-1:
              ax = k_grid[m-1]
              bx = k_grid[m] 
              cx = k_grid[m] 
            

          if ax==bx:  # corner solution: a'=0?
              # w = wage_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
              # r = interest_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
              # T = transfer(K_prev = k_grid[m], H = H_bar, L = L_bar, G = grants, tau = tau, w = w, r = r)
              if value_function_parent_final(next_value_interpolator = vr_polate, k = ax, k_prev = k0, h = hc0, w = w, r = r, tau = tau, T = T, hc = hc0, I_e = Ie, e_mean = e_mean[0], I_e_mean = I_e_mean) > value_function_parent_final(next_value_interpolator = vr_polate, k = k_grid[m], k_prev = k_eps, h = h0, w = w, r = r, tau = tau, T = T, hc = hc0, I_e = Ie, e_mean = e_mean[0], I_e_mean = I_e_mean):
                k13d_optimal_array[ik, ih, i-1, ihc]=0
              else:
                # k_optimal_array[j,i-1] = Search(value_function, left=ax, initial_guess=k_eps, right=cx, tolerance=tolerance_gs, k_prev=k0, l=l, h=h, w=w, r=r, tau=tau, T=T)
                k13d_optimal_array[ik, ih, i-1, ihc] = GSS2(vr_polate, value_function_parent_final, ax, k_eps, cx, tolerance_gs,
                    k_prev=k0, h=h0, w=w, r=r, tau=tau, T=T, hc=hc0, I_e=Ie, e_mean=e_mean[0], I_e_mean=I_e_mean)
                    
          elif bx==cx:  # corner solution: a'=a[na-1]=kmax?
              # w = wage_rate(K_prev = k_grid[na-1], H = H_bar, L = L_bar)
              # r = interest_rate(K_prev = k_grid[na-1], H = H_bar, L = L_bar)
              # T = transfer(K_prev = k_grid[na-1], H = H_bar, L = L_bar, G = grants, tau = tau, w = w, r = r)
              if value_function_parent_final(next_value_interpolator = vr_polate, k = k_grid[na-1], k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T, hc = hc0, I_e = Ie, e_mean = e_mean[0], I_e_mean = I_e_mean) > value_function_parent_final(vr_polate, k = k_grid[na-1] - k_eps, k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T, hc = hc0, I_e = Ie, e_mean = e_mean[0], I_e_mean = I_e_mean):
                k13d_optimal_array[ik, ih, i-1] = k_grid[na-1]
              else:
                # k_optimal_array[j,i-1] = Search(value_function, left=k_grid[na-2], initial_guess=k_max-k_eps, right=k_max, tolerance=tolerance_gs,
                #     k_prev=k0, l=l, h=h, w=w, r=r, tau=tau, T=T)  
                k13d_optimal_array[ik, ih, i-1, ihc] = GSS2(vr_polate, value_function_parent_final, k_grid[na-2], k_max-k_eps, k_max, tolerance_gs,
                    k_prev=k0, h=h0, w=w, r=r, tau=tau, T=T, hc=hc0, I_e=Ie, e_mean=e_mean[0], I_e_mean=I_e_mean)    
            
          else:
              # w = wage_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
              # r = interest_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
              # T = transfer(K_prev = k_grid[m], H = H_bar, L = L_bar, G = grants, tau = tau, w = w, r = r)
              # k_optimal_array[j,i-1] = Search(value_function, left=ax, initial_guess=bx, right=cx, tolerance=tolerance_gs,
              #       k_prev=k0, l=l, h=h, w=w, r=r, tau=tau, T=T)
              k13d_optimal_array[ik,ih, i-1, ihc] = GSS2(vr_polate, value_function_parent_final, ax, bx, cx, tolerance_gs,
                    k_prev=k0, h=h0, w=w, r=r, tau=tau, T=T, hc=hc0, I_e=Ie, e_mean=e_mean[0], I_e_mean=I_e_mean)
              
                

          k1 = k13d_optimal_array[ik, ih, i-1, ihc]
          v3d_optimal_array[ik, ih, i-1, ihc] = value_function_parent_final(vr_polate, k13d_optimal_array[ik, ih, i-1, ihc], k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T, hc=hc0, I_e=Ie, e_mean=e_mean[0], I_e_mean=I_e_mean)
        # l_optimal_array[ik, ih, i-1] = labor_foc(k = k1, k_prev = k0, h = h, w = w, r = r, tau = tau, T = T)
        # c_optimal_array[ik,ih, i-1] = (1-tau)*w*h*l_optimal_array[ik, ih, i-1] + r*k0 - tau*r*max(k0, 0) + T - k1
        # # print(f"iter: {iteration}, i: {i}, k opt: {k1}")


  print("Got to checkpoint")
  #compute policy function for period 3, 2, 1 adult
  for i in range(child_periods-1, 0, -1): #loop backwards from age 60 to 0
    for ihc in range(nh):
      for ih in range(nh):
        vr_polate = interpolate.interp1d(k_grid, v3d_optimal_array[:,ih,i,ihc], kind='cubic', fill_value='extrapolate') #interpolator which lines up k_grid with value_array row i
        h0 = h_grid[ih]
        hc0 = h_grid[ihc]
        m0 = 0

        for ik in range(na): #iteration through all possible kt-1 assets could come into periods with
          # print(f"backwards iter: {iteration}, solving for age: {i}, k_t-1: {k_grid[ik]}, h_t: {h_grid[ih]}")

          k0 = k_grid[ik]

          ax = 0
          bx = -1
          cx = -2
          v0 = neg
          m = max(-1, m0-2)

          while (ax > bx or bx > cx):
            m += 1
            v1 = value_function_parent(next_value_interpolator = vr_polate, k = k_grid[m], k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T, hc = hc0, I_e = Ie, e_mean = e_mean[0], I_e_mean = I_e_mean)

            if v1 > v0:
              if m == 0:
                ax = k_grid[m]
                bx = k_grid[m]

              else:
                bx = k_grid[m]
                ax = k_grid[m-1]

              v0 = v1
              m0 = m

            else:
              cx = k_grid[m]

            if m==na-1:
              ax = k_grid[m-1]
              bx = k_grid[m] 
              cx = k_grid[m] 
            

          if ax==bx:  # corner solution: a'=0?
              # w = wage_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
              # r = interest_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
              # T = transfer(K_prev = k_grid[m], H = H_bar, L = L_bar, G = grants, tau = tau, w = w, r = r)
              if value_function_parent(next_value_interpolator = vr_polate, k = ax, k_prev = k0, h = hc0, w = w, r = r, tau = tau, T = T, hc = hc0, I_e = Ie, e_mean = e_mean[0], I_e_mean = I_e_mean) > value_function_parent(next_value_interpolator = vr_polate, k = k_grid[m], k_prev = k_eps, h = h0, w = w, r = r, tau = tau, T = T, hc = hc0, I_e = Ie, e_mean = e_mean[0], I_e_mean = I_e_mean):
                k13d_optimal_array[ik, ih, i-1, ihc]=0
              else:
                # k_optimal_array[j,i-1] = Search(value_function, left=ax, initial_guess=k_eps, right=cx, tolerance=tolerance_gs, k_prev=k0, l=l, h=h, w=w, r=r, tau=tau, T=T)
                k13d_optimal_array[ik, ih, i-1, ihc] = GSS2(vr_polate, value_function_parent, ax, k_eps, cx, tolerance_gs,
                    k_prev=k0, h=h0, w=w, r=r, tau=tau, T=T, hc=hc0, I_e=Ie, e_mean=e_mean[0], I_e_mean=I_e_mean)
                    
          elif bx==cx:  # corner solution: a'=a[na-1]=kmax?
              # w = wage_rate(K_prev = k_grid[na-1], H = H_bar, L = L_bar)
              # r = interest_rate(K_prev = k_grid[na-1], H = H_bar, L = L_bar)
              # T = transfer(K_prev = k_grid[na-1], H = H_bar, L = L_bar, G = grants, tau = tau, w = w, r = r)
              if value_function_parent(next_value_interpolator = vr_polate, k = k_grid[na-1], k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T, hc = hc0, I_e = Ie, e_mean = e_mean[0], I_e_mean = I_e_mean) > value_function_parent(vr_polate, k = k_grid[na-1] - k_eps, k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T, hc = hc0, I_e = Ie, e_mean = e_mean[0], I_e_mean = I_e_mean):
                k13d_optimal_array[ik, ih, i-1] = k_grid[na-1]
              else:
                # k_optimal_array[j,i-1] = Search(value_function, left=k_grid[na-2], initial_guess=k_max-k_eps, right=k_max, tolerance=tolerance_gs,
                #     k_prev=k0, l=l, h=h, w=w, r=r, tau=tau, T=T)  
                k13d_optimal_array[ik, ih, i-1, ihc] = GSS2(vr_polate, value_function_parent, k_grid[na-2], k_max-k_eps, k_max, tolerance_gs,
                    k_prev=k0, h=h0, w=w, r=r, tau=tau, T=T, hc=hc0, I_e=Ie, e_mean=e_mean[0], I_e_mean=I_e_mean)    
            
          else:
              # w = wage_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
              # r = interest_rate(K_prev = k_grid[m], H = H_bar, L = L_bar)
              # T = transfer(K_prev = k_grid[m], H = H_bar, L = L_bar, G = grants, tau = tau, w = w, r = r)
              # k_optimal_array[j,i-1] = Search(value_function, left=ax, initial_guess=bx, right=cx, tolerance=tolerance_gs,
              #       k_prev=k0, l=l, h=h, w=w, r=r, tau=tau, T=T)
              k13d_optimal_array[ik,ih, i-1, ihc] = GSS2(vr_polate, value_function_parent, ax, bx, cx, tolerance_gs,
                    k_prev=k0, h=h0, w=w, r=r, tau=tau, T=T, hc=hc0, I_e=Ie, e_mean=e_mean[0], I_e_mean=I_e_mean)
              
                

          k1 = k13d_optimal_array[ik, ih, i-1, ihc]
          v3d_optimal_array[ik, ih, i-1, ihc] = value_function_parent(vr_polate, k13d_optimal_array[ik, ih, i-1, ihc], k_prev = k0, h = h0, w = w, r = r, tau = tau, T = T, hc=hc0, I_e=Ie, e_mean=e_mean[0], I_e_mean=I_e_mean)
        # l_optimal_array[ik, ih, i-1] = labor_foc(k = k1, k_prev = k0, h = h, w = w, r = r, tau = tau, T = T)
        # c_optimal_array[ik,ih, i-1] = (1-tau)*w*h*l_optimal_array[ik, ih, i-1] + r*k0 - tau*r*max(k0, 0) + T - k1
        # # print(f"iter: {iteration}, i: {i}, k opt: {k1}")
  print("Got to checkpoint")


  
  # Display the full arrays without truncation
  # np.set_printoptions(threshold=np.inf)
  # print("\nFull output of k1_optimal_array[0]:")
  # print(k1_optimal_array[0])
  # print("\nFull output of k1_optimal_array[10]:")
  # print(k1_optimal_array[10])
  # # Reset print options to default
  # np.set_printoptions(threshold=1000)

  iteration_ss = -1
  diff_ss = 99
  h0sim = 1
  max_iters_ss = 30
  hp = h0sim
  tolerance_ss = 0.005
  hc1 = 1

  # k_gen = np.zeros(periods + 1)
  # l_gen = np.zeros(periods)
  # c_gen = np.zeros(periods)
  # hc_gen = np.zeros(child_periods + 1)
  # i_gen = np.zeros(periods)

  while iteration_ss < max_iters_ss - 1 and diff_ss > tolerance_ss:
    iteration_ss += 1
    
    k_gen = np.zeros(periods + 1)
    l_gen = np.zeros(periods)
    c_gen = np.zeros(periods)
    hc_gen = np.zeros(child_periods + 1)
    i_gen = np.zeros(periods)


    k_gen[0] = k_start
    hc_gen[0] = hc1

    for q in range(periods):
      bi_polate = interpolate.RegularGridInterpolator((k_grid, h_grid), k1_optimal_array[:,:,q], method='linear', bounds_error=False, fill_value=None) #bilinear interpolator using k_grid and h_grid

      if q < periods - 1:
        k_gen[q+1] = float(bi_polate(np.array([[k_gen[q], hp]]))) #right index?
      else:
        k_gen[q+1] =0
      
      
      l_gen[q] = labor_foc(k = k_gen[q+1], k_prev = k_gen[q], h = hp, w = w, r = r, tau = tau, T = T)
      c_gen[q] = consumption(k_prev=k_gen[q], k=k_gen[q+1], h=hp, l=l_gen[q], w=w, r=r, tau=tau, T=T)
      i_gen[q] = -c_gen[q] + (1-tau)*w*hp*l_gen[q] + (r+delta)*k_gen[q] - tau*(r+delta)*max(k_gen[q], 0) + T


      #child concieved period 0 and born period 1
      if q == 0:
        hc_gen[1] = hc_gen[0]

      if q < child_periods and q > 0:
        #could have phi and everything vary here

        log_h = np.log(hp) + 1
        e_p = ((omega * log_h) * (1-l_gen[q])) / (1 + omega * log_h)
        hc_gen[q+1] = human_capital_formation(h_c = hc_gen[q], e_p = e_p, I_e = Ie, e_mean = e_mean[q], I_e_mean = I_e_mean)
        print(f"age: {q}, ep: {e_p}, hc: {hc_gen[q+1]}")

    print(f"Ie: {Ie}")
    final_period_assets = (1-tau)*w*hp*l_gen[periods-1] + (1+r)*k_gen[periods-2] - tau*r*max(k_gen[periods-2], 0) + T
    c_gen[periods-1] = final_period_assets

    # print(hc_gen)
    diff_ss = abs((hp - hc_gen[child_periods])/hp)
    print(f"diff_ss: {diff_ss}, h: {hp}, hc_gen[child_periods]: {hc_gen[child_periods]}")
    hp = hc_gen[child_periods] #check to see if right

    #could shock h each period 

    #add distance h and iter increment


  K_new = np.mean(k_gen[:periods-1])
  L_new = np.mean(l_gen)
  H_new = hp

  K_bar = phi*K_old + (1-phi)*K_new
  H_bar = phi*H_old + (1-phi)*H_new
  Z_bar = (1-delta)*Z_old + (1-sigma_bar)*Iz
  L_bar = phi*L_old + (1-phi)*L_new




  print("Solution for aggregate capital stock K: " + str(K_bar))
  print("Solution for aggregate labor N: " + str(L_bar))


  fig, axes = plt.subplots(2, 2, figsize=(12, 8))
  axes[0, 0].set_xlabel('age')
  axes[0, 0].set_ylabel('physical capital entering period t (k_t)')
  axes[0, 0].plot(k_gen)
  axes[0, 0].grid(True)

  axes[0, 1].set_xlabel('age')
  axes[0, 1].set_ylabel('consumption (c) and investment (I)')
  axes[0, 1].plot(c_gen, label='consumption (c_t)')
  axes[0, 1].plot(i_gen[:-1], label='net investment (I_t)')
  axes[0, 1].legend()
  axes[0, 1].grid(True)

  axes[1, 0].set_xlabel('age')
  axes[1, 0].set_ylabel('labor supply (l_t)')
  axes[1, 0].plot(l_gen)
  axes[1, 0].grid(True)

  axes[1, 1].set_xlabel('age')
  axes[1, 1].set_ylabel('human capital (hc_t)')
  axes[1, 1].plot(hc_gen)
  axes[1, 1].grid(True)


  
  plt.show()     


    #update H, L


iter: 0; K change: 151.23; H change: 50.00; L change: 6.67; Z change: 1000.00
y: 0.5397986988643104
ie: 0.032387921931858625
tax_revenue: 0.05485694361172334
grants: 0.03778590892050173
T: 0.027867008668507817
iter: 0; K: 0.66; H: 2.00; L: 0.30; Z: 0.10
iter: 0; Y: 0.54; tau: 0.15; T: 0.03


/var/folders/w4/8bdyf7817wzghq2ypzxtqpcm0000gn/T/ipykernel_29639/3499442841.py:16: RuntimeWarning: invalid value encountered in scalar divide
  w1 = dt[1] - (dt[0]**2 / dt[1])
/var/folders/w4/8bdyf7817wzghq2ypzxtqpcm0000gn/T/ipykernel_29639/3499442841.py:17: RuntimeWarning: invalid value encountered in scalar divide
  wr = dt[r-1] - (dt[r]**2 / dt[r-1])
/var/folders/w4/8bdyf7817wzghq2ypzxtqpcm0000gn/T/ipykernel_29639/3499442841.py:30: RuntimeWarning: invalid value encountered in scalar divide
  T[0, 1] = dt[0] + (dt[0]**2 / dt[1])
/var/folders/w4/8bdyf7817wzghq2ypzxtqpcm0000gn/T/ipykernel_29639/3499442841.py:32: RuntimeWarning: invalid value encountered in scalar divide
  T[r-1, r-2] = dt[r] + (dt[r]**2 / dt[r-1])


Got to checkpoint


KeyboardInterrupt: 

aggregates

In [15]:
print("Aggregate Factors of Production:")
print(f"Y: {Y:.4f}")
print(f"Z: {Z_old:.4f}")
print(f"K: {K_old:.4f}")
print(f"H: {H_old:.4f}")
print(f"L: {L_old:.4f}")
print("\nRates:")
print(f"Interest rate (r): {r:.4f}")
print(f"Return on capital (r^k): {r + delta:.4f}")


Aggregate Factors of Production:
Y: 0.4982
Z: 0.1852
K: 0.2828
H: 2.4784
L: 0.2660

Rates:
Interest rate (r): 0.3052
Return on capital (r^k): 0.4052
